# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [1]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des politiciens belges

In [2]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death
statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P106 wd:Q82955 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Belgian politicians found\n")
print(rows[:10])


7712 Belgian politicians found

[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17350686'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abdallah Kanfaoui'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1968-05-21T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13461801'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abdelaziz Charkaoui Hajri'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1951-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q27903935'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abderrahim Lahlali'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1978-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3294738'}, 'person

## Filtrer pour n'afficher que les noms contenant "Bouchez"

In [3]:
name_filter = 'Bouchez'
max_results = 10

date_format = "%Y-%m-%dT%H:%M:%SZ"
filtered_rows = [row for row in rows if name_filter in row['personLabel']['value']]
print(f"Displaying the first {max_results}:\n")
for row in filtered_rows[:max_results]:
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format)
        birth_year = birth_date.year
    except ValueError:
        birth_year = "????"
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format)
        death_year = death_date.year
    except ValueError: # unknown death date
        death_year = "????"
    except KeyError: # still alive
        death_year = ""
    print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

Displaying the first 10:

Georges-Louis Bouchez (1986-)


## Pour en savoir plus

- Le projet Wikidata : https://www.wikidata.org/wiki/Wikidata:Main_Page
- Aide à la construction de requêtes : https://query.wikidata.org/